<a href="https://colab.research.google.com/github/NahuelRepetto/Programacion-Concurrente/blob/main/TP2_P1_OpenMP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Programacion Concurrente - TP2 - Parte 1

Utilizamos 2 prácticas con OpenMP desarrolladas en lenguaje C/C++ nativo en plataforma colab. 


---
## 1.1. Ejercicio 1 - Hola Mundo estilo Programacion Concurrente


### 1.1.1. Preguntas ejercicio 1:

a) Identifique los 3 componentes de openMP en el ejercicio 1.

b) Realice pruebas modificando la cantidad de hilos (1, 3, 7). Que conclusiones pudo determinar?.

c) ¿Cuál es la diferencia las formas de asignación *static* y *dynamic* en la cláusula *schedule*?, ¿Qué sucede si utiliza *dynamic* en el código?

d) En el for: ¿Que sucede con el valor de j, sí se parametriza como lastprivate?


### Respuestas:

a) Podemos identificar rápidamente los componentes de openMP gracias a la sintaxis especifica que se utiliza para llamar a estas directivas: **#pragma omp …**

Primeramente, encontramos el uso de la directiva parallel para definir una región paralela, es decir, el código encerrado por dicha sección será ejecutado por múltiples subprocesos en paralelo.
Luego, dentro de esta encontramos 3 directivas más:
-	**Single** para especificar que esa sección de código debe ejecutarse en un solo subproceso, no necesariamente en el subproceso principal.
-	**Master** para especifica que solo el subproceso principal debe ejecutar una sección del programa.
-	**for schedule(…)**para dividir el trabajo realizado por el ciclo for que está dentro región paralela entre varios hilos.


b) La cantidad de hilos creados va a influir en la ejecución ciclo for.
Por como esta definido este ciclo **for(i=0;i<20;i++)** sabemos que iterara 20 veces, y por como esta definida la directiva de openMP **#pragma omp for schedule(static, 5)** podemos deducir que dividirá estas 20 iteraciones bloques de 4 iteraciones cada una. Dicho esto, podemos afirmar que según la cantidad de hilos que se creen puede que haya algunos que estén ociosos. Si tenemos más hilos que bloques sobre los que iterar, la cantidad sobrante de hilos no harán nada. Y si tenemos menos hilos que bloques sobre los que iterar, unos o mas hilos iterara sobre más de un bloque.
También cabe destacar que, como trabajamos con 2 procesadores, realmente tendremos solo dos hilos trabajando paralelamente, esto no quita que no podamos crear más de dos hilos, podemos crear la cantidad que queramos, pero esto no aumentara realmente el paralelismo.


c) El tipo de Schedule **Static** divide el total de bloques por el numero enviado por parámetros, creando así secciones de n cantidad de bloques de los cuales se hace cargo un hilo por sección. En el ejemplo de este programa la cantidad total de bloques seria 20, y al enviar por parámetros un 5, se dividirá en 5 secciones de 4 bloques cada una. 

El Schedule **Dynamic** por otro lado, en lugar de dividir los bloques por cierta cantidad y asignárselo directamente a un hilo, le va dando un bloque sobre el que operar a aquellos hilos que estén disponibles.

Al usar el Dynamic en el código se aprecia que no todos los hilos participan, en general se distribuyen los bloques entre 2 o 3 hilos, y en ocasiones solo 1 hilo hace todo el trabajo.
Esto no es de extrañar ya que en verdad solo hay dos hilos en simultaneo, debido a que solo hay dos procesadores reales, por lo que a medida que terminan de hacer una tarea ya le asigna otra, no es que realmente haya 3, 4, 5, 6… hilos simultáneos, si fuera así si veríamos esta cantidad de hilos dividiéndose las operaciones. 


d) La función del **lastPrivate** es que, toma el valor que la variable tiene dentro de la última iteración y se lo asigna a la variable original. 
En este programa incluir el lastPrivate para la variable j causa un mal funcionamiento, arrojando al final como valor de j un número indeterminado. Por como el lastPrivate interpreto que este mal funcionamiento se debe a que el trabajo que se realiza en cada hilo es el de ir incrementando el valor de j del main, pero no se le asigna un valor dentro de esa iteración.
Mientras que el lastPrivate toma el valor que una variable tiene asignada en la última iteración, y para las variables a las que no se les asigna un valor, tienen valores indeterminados.

### 1.1.2. Código Lenguaje C 

In [112]:
%%writefile code.cpp

// Hola Mundo desde OpenMP, usando c, ejecutado en Colab. 

#include <iostream>
#include <vector>
#include <omp.h>    // Cabecera OpenMP   
  
int main(int argc, char* argv[]) 
{
  int i,j=0;
  std::cout<<"Inicio..."<<std::endl;

  //-----------------------------------------------------------------------------------------------
  // Region paralela
  #pragma omp parallel 
  { 
    std::cout<<"Hola mundo!!!... soy el hilo " << omp_get_thread_num()<< ", de "<<  omp_get_num_threads() <<", procesadores "<< omp_get_num_procs()<< std::endl;  

    #pragma omp single
    {
      std::cout<<"Hola mundo!!!... soy el hilo " << omp_get_thread_num()<< " uno de muchos."<< std::endl;
    }
    #pragma omp master
    {
      std::cout<<"Hola mundo!!!... soy el hilo maestro: " << omp_get_thread_num()<< std::endl;
    }

    #pragma omp for schedule(dynamic, 5)
    for(i=0;i<20;i++)
    {
      j++;
      std::cout<<"Hola mundo!!!... soy el hilo " << omp_get_thread_num()<< ", itero para i ."<<i<<", actualizo j "<<j<< std::endl;
    }
  }
  // Region paralela
  //-----------------------------------------------------------------------------------------------

  std::cout<<"Fin..., con j="<<j<<std::endl;
}

Overwriting code.cpp


### 1.1.3 Compilación de Hola Mundo. 

In [113]:
!g++ -o hello -fopenmp code.cpp

### 1.1.4. Ejecución Hola Mundo en OpenMP.

In [137]:
%env OMP_NUM_THREADS=3
!./hello

env: OMP_NUM_THREADS=3
Inicio...
Hola mundo!!!... soy el hilo 0, de 3, procesadores 2
Hola mundo!!!... soy el hilo 0 uno de muchos.
Hola mundo!!!... soy el hilo 2, de 3, procesadores 2
Hola mundo!!!... soy el hilo 1, de 3, procesadores 2
Hola mundo!!!... soy el hilo maestro: 0
Hola mundo!!!... soy el hilo 0, itero para i .0, actualizo j 1
Hola mundo!!!... soy el hilo 0, itero para i .1, actualizo j 2
Hola mundo!!!... soy el hilo 0, itero para i .2, actualizo j 3
Hola mundo!!!... soy el hilo 0, itero para i .3, actualizo j 4
Hola mundo!!!... soy el hilo 0, itero para i .4, actualizo j 5
Hola mundo!!!... soy el hilo 0, itero para i .5, actualizo j 6
Hola mundo!!!... soy el hilo 0, itero para i .6, actualizo j 7
Hola mundo!!!... soy el hilo 0, itero para i .7, actualizo j 8
Hola mundo!!!... soy el hilo 0, itero para i .8, actualizo j 9
Hola mundo!!!... soy el hilo 0, itero para i .9, actualizo j 10
Hola mundo!!!... soy el hilo 0, itero para i .10, actualizo j 11
Hola mundo!!!... soy el hi

---
## 1.2. Ejercicio 2 - Axpy

La función Axpy es parte de las bibliotecas BLAS [3]. Ella se encarga de realizar la suma de vectores, con la siguiente ecuación:

<center>$Y = \alpha X + Y$</center>

En donde:
> $\alpha$: Es un escala.

> $X$: Es el vector origen.

> $Y$: Es el vector destino.


### 1.2.1 Preguntas Ejercicio 2
a) Ejecute el ejercicio 1.2.2:

    - Con la variable cantidad_N para: 50, 500, 5.000.
    - Con los valores hilos_ 2, 4, 6.

b) Repita la prueba (a), con ciclos 1, 10, 30. ¿Cuál fue la diferencia?

c) ¿Por qué el SpeedUp no mejora a medida que aumentan la cantidad de hilos?

d) ¿Qué sucede con la eficiencia a medida que aumentan la cantidad_N?¿Porqué no llega al valor ideal?.

### 1.2.2 Código Lenguaje C 

In [138]:
%%writefile code_axpy.cpp
// Axpy con OpenMP, usando c, ejecutado en Colab. 

#include <iostream>
#include <vector>
#include <cstdlib>
#include <sys/time.h>
#include <omp.h>    // Cabecera OpenMP   

// ----------------------------------------------------------------------------
// Macros que miden el tiempo.

static double dHashTiempoHistory[3];
static struct timeval tv;

#define TIEMPO_INI( h )      \
   gettimeofday(&tv,NULL);   \
   dHashTiempoHistory[ h ] = tv.tv_sec + tv.tv_usec/1000000.0;
   
   
#define TIEMPO_FIN( h )      \
   gettimeofday(&tv,NULL);   \
   dHashTiempoHistory[ h ] = ((tv.tv_sec + tv.tv_usec/1000000.0) - dHashTiempoHistory[ h ]) * 1000; // Devuelvo en milisegundos
#define TIEMPO_GET( h ) dHashTiempoHistory[ h ]

#define HTH_TOTAL         1
#define HTH_AXPY_SEC      2
#define HTH_AXPY_OMP      3

using namespace std;

// ----------------------------------------------------------------------------

void axpy( double alfa, vector<double> &X, vector<double> &Y )
{
  int i;

  #pragma omp parallel for
  for(i=0;i<Y.size();i++)
  {
    Y[i] = alfa * X[i] + Y[i];
  }
}

// ----------------------------------------------------------------------------

int main(int argc, char* argv[]) 
{ 
  int i,c;
  TIEMPO_INI( HTH_TOTAL )

  // Leo los parametros.
  if( argc != 4 )
  {
      std::cerr<< " Error en los parametros de indicar: (alfa), (Tamanio vector), (ciclos ejecucion)."<<argc<<std::endl;
      exit( -1 );
  }

  float alfa     = atof( argv[1] );
  int cantidad_N = atoi( argv[2] );
  int ciclos     = atoi( argv[3] );

  // --------------------------------------------
  // Defino la memoria de los vectores.

  vector<double> X( cantidad_N );
  vector<double> Y( cantidad_N );

  for (int i=0;i<X.size();i++)
  {
    X[i] = (rand()/(double)RAND_MAX)*0.73;
    Y[i] = (rand()/(double)RAND_MAX)*0.71;
  }

  // --------------------------------------------
  // Realizo la función Axpy con OpenMP.

  TIEMPO_INI( HTH_AXPY_OMP )

  for(c=0;c<ciclos;c++)
  {
    axpy( alfa, X, Y );
  }

  TIEMPO_FIN( HTH_AXPY_OMP )

  // --------------------------------------------
  // Realizo la función Axpy en forma secuencial.

  omp_set_num_threads(1);

  TIEMPO_INI( HTH_AXPY_SEC )

  for(c=0;c<ciclos;c++)
  {
    axpy( alfa, X, Y );
  }

  TIEMPO_FIN( HTH_AXPY_SEC )

  TIEMPO_FIN( HTH_TOTAL )

 std::cout<<"Valores Reales  :" <<std::endl;
 std::cout<<"Tiempo TOTAL     : "<<TIEMPO_GET(HTH_TOTAL   )<<" [ms]"<<std::endl;
 std::cout<<"Tiempo axpy Sec  : "<<TIEMPO_GET(HTH_AXPY_SEC)<<" [ms]"<<std::endl;
 std::cout<<"Tiempo axpy Omp  : "<<TIEMPO_GET(HTH_AXPY_OMP)<<" [ms]"<<std::endl;
 std::cout<<std::endl;
 std::cout<<"SpeedUp          : (tiempo Secuencial/tiempo paralelo) : "<<TIEMPO_GET(HTH_AXPY_SEC)<<" / "<<TIEMPO_GET(HTH_AXPY_OMP)<<" = "<<TIEMPO_GET(HTH_AXPY_SEC)/TIEMPO_GET(HTH_AXPY_OMP)<<std::endl;
 std::cout<<"Eficiencia       : SpeedUp/nro procesadores            : "<<TIEMPO_GET(HTH_AXPY_SEC)/TIEMPO_GET(HTH_AXPY_OMP)<<" / "<<omp_get_num_procs()<<" = "<<TIEMPO_GET(HTH_AXPY_SEC)/(omp_get_num_procs()*TIEMPO_GET(HTH_AXPY_OMP))<<std::endl;
 std::cout<<"Coste Sec        : nro procesadores*Tiempo             : "<<1<<" * "<<TIEMPO_GET(HTH_AXPY_SEC)<<" = "<<TIEMPO_GET(HTH_AXPY_SEC)<<std::endl;
 std::cout<<"Coste Omp        : nro procesadores*Tiempo             : "<<omp_get_num_procs()<<" * "<<TIEMPO_GET(HTH_AXPY_OMP)<<" = "<<omp_get_num_procs()*TIEMPO_GET(HTH_AXPY_OMP)<<std::endl;
 std::cout<<"Funcion Overhead : Coste Omp - tiempo Secuencial       : "<<omp_get_num_procs()*TIEMPO_GET(HTH_AXPY_OMP)<<" - "<<TIEMPO_GET(HTH_AXPY_SEC)<<" = "<<(omp_get_num_procs()*TIEMPO_GET(HTH_AXPY_OMP))-TIEMPO_GET(HTH_AXPY_SEC)<<std::endl;


 std::cout<<std::endl;
 std::cout<<"Valores Ideal: "<<std::endl;
 TIEMPO_GET(HTH_AXPY_OMP) = TIEMPO_GET(HTH_AXPY_SEC) / omp_get_num_procs();
 std::cout<<"Tiempo axpy Sec  : "<<TIEMPO_GET(HTH_AXPY_SEC)<<" [ms]"<<std::endl;
 std::cout<<"Tiempo axpy Omp  : "<<TIEMPO_GET(HTH_AXPY_OMP)<<" [ms]"<<std::endl;

 std::cout<<"SpeedUp          : (tiempo Secuencial/tiempo paralelo) : "<<TIEMPO_GET(HTH_AXPY_SEC)<<" / "<<TIEMPO_GET(HTH_AXPY_OMP)<<" = "<<TIEMPO_GET(HTH_AXPY_SEC)/TIEMPO_GET(HTH_AXPY_OMP)<<std::endl;
 std::cout<<"Eficiencia       : SpeedUp/nro procesadores            : "<<TIEMPO_GET(HTH_AXPY_SEC)/TIEMPO_GET(HTH_AXPY_OMP)<<" / "<<omp_get_num_procs()<<" = "<<TIEMPO_GET(HTH_AXPY_SEC)/(omp_get_num_procs()*TIEMPO_GET(HTH_AXPY_OMP))<<std::endl;
 std::cout<<"Coste Sec        : nro procesadores*Tiempo             : "<<1<<" * "<<TIEMPO_GET(HTH_AXPY_SEC)<<" = "<<TIEMPO_GET(HTH_AXPY_SEC)<<std::endl;
 std::cout<<"Coste Omp        : nro procesadores*Tiempo             : "<<omp_get_num_procs()<<" * "<<TIEMPO_GET(HTH_AXPY_OMP)<<" = "<<omp_get_num_procs()*TIEMPO_GET(HTH_AXPY_OMP)<<std::endl;
 std::cout<<"Funcion Overhead : Coste Omp - tiempo Secuencial       : "<<omp_get_num_procs()*TIEMPO_GET(HTH_AXPY_OMP)<<" - "<<TIEMPO_GET(HTH_AXPY_SEC)<<" = "<<(omp_get_num_procs()*TIEMPO_GET(HTH_AXPY_OMP))-TIEMPO_GET(HTH_AXPY_SEC)<<std::endl;


}
// ----------------------------------------------------------------------------

Writing code_axpy.cpp


### 1.2.3. Compilación de código C Axpy. 

In [139]:
!g++ -o axpy -fopenmp code_axpy.cpp

### 1.2.4. Ejecución Axpy.

In [ ]:
# --------------------------------------------
#@title Parámetros de ejecución { vertical-output: true }

Cantidad_N = 50#@param {type: "number"}
Alfa       = 1.0#@param {type: "number"}
Ciclos     = 10#@param {type: "slider", min: 10, max: 200}
Hilos      = 2#@param {type: "slider", min: 1, max: 10}

# --------------------------------------------

%env OMP_NUM_THREADS=$Hilos
!./axpy $Alfa $Cantidad_N $Ciclos

---
# Medidas de prestaciones en algoritmos paralelos
Las técnicas de HPC buscan reducir los tiempos de ejecución, el tiempo como medida, no alcanza. Dos algoritmos pueden ejecutar en el mismo tiempo, pero uno de ellos usa menos procesadores [6,7]. 


## SpeedUp
Referencia a la ganancia de velocidad que se consigue con un algoritmo paralelo, al resolver el mismo problema con respecto al algoritmo secuencial.

<center>$ SpeedUp = \frac{Tiempo\_Secuencial}{Tiempo\_Paralelo} $</center>



##Eficiencia
La eficiencia normaliza el valor del SpeedUp, al dividirlo por la cantidad de procesadores que se utilizaron para alcanzar la ganancia en velocidad. Dando la idea de la porción de tiempo que los procesadores se dedican al trabajo útil.

<center>$ Eficiencia = \frac{SpeedUp}{Nro\_Procesadores} $</center>


## Coste
El coste de un algoritmo paralelo representa el tiempo realizado por todo el sistema en la resolución del problema.

<center>$ coste = Nro\_Procesadores \times Tiempo\_Algoritmo$</center>


## Función Overhead
Es la diferencia entre el Coste y el tiempo secuencial. Mientras mayor es la función overhead, peor es el comportamiento del algoritmo paralelo.

<center>$ Overhead = {Coste}-{Tiempo\_Secuencial} $</center>

---

In [ ]:
!cat /proc/cpuinfo

---
# Bibliografía
[1] Función Axpy de biblioteca BLAS: [Referencia](https://software.intel.com/content/www/us/en/develop/documentation/mkl-developer-reference-c/top/blas-and-sparse-blas-routines/blas-routines/blas-level-1-routines-and-functions/cblas-axpy.html)

[3] Biblioteca BLAS: [Referencia](http://www.netlib.org/blas/)

[4] F. Almeida, D. Gimenéz, A. Vidal - Introducción a la programación paralela - 2008 - Editorial Parafino.

[5] D. Jiménez-González - Introducción a las arquitecturas paralelas. [PDF](http://progc-unlam.com.ar/material-clase/OpenMP-MPI/Introducci%C3%B3n%20a%20la%20Computaci%C3%B3n%20Paralela.pdf)